In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

 

## Time series

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.metrics import r2_score

<br><br>

## Exercise 1: time series prediction

 We'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture, we had different measurements for each Location. How about this avocado sales dataset? For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    

Similar to the "Rain in Australia" dataset from lecture we have different locations (54 locations) along with the Date, thus it is likely that we will do separate date measurements. Given that there could be repeated dates for different locations. 

In [9]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

In [10]:
df['region'].unique().shape

(54,)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          18249 non-null  datetime64[ns]
 1   AveragePrice  18249 non-null  float64       
 2   Total Volume  18249 non-null  float64       
 3   4046          18249 non-null  float64       
 4   4225          18249 non-null  float64       
 5   4770          18249 non-null  float64       
 6   Total Bags    18249 non-null  float64       
 7   Small Bags    18249 non-null  float64       
 8   Large Bags    18249 non-null  float64       
 9   XLarge Bags   18249 non-null  float64       
 10  type          18249 non-null  object        
 11  year          18249 non-null  int64         
 12  region        18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 


In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    

As we see from the code in the last cell the every date recorded can be expressed as a week away from the date previous, thus all the data is equally spaced.

In [12]:
df['Date'].unique()

<DatetimeArray>
['2015-12-27 00:00:00', '2015-12-20 00:00:00', '2015-12-13 00:00:00',
 '2015-12-06 00:00:00', '2015-11-29 00:00:00', '2015-11-22 00:00:00',
 '2015-11-15 00:00:00', '2015-11-08 00:00:00', '2015-11-01 00:00:00',
 '2015-10-25 00:00:00',
 ...
 '2018-03-11 00:00:00', '2018-03-04 00:00:00', '2018-02-25 00:00:00',
 '2018-02-18 00:00:00', '2018-02-11 00:00:00', '2018-02-04 00:00:00',
 '2018-01-28 00:00:00', '2018-01-21 00:00:00', '2018-01-14 00:00:00',
 '2018-01-07 00:00:00']
Length: 169, dtype: datetime64[ns]

In [13]:
df['Date']    

0    2015-12-27
1    2015-12-20
2    2015-12-13
3    2015-12-06
4    2015-11-29
        ...    
7    2018-02-04
8    2018-01-28
9    2018-01-21
10   2018-01-14
11   2018-01-07
Name: Date, Length: 18249, dtype: datetime64[ns]

In [14]:
start_date = df["Date"].min()
end_date = df["Date"].max()
df_sorted = df.sort_values(by='Date', ascending=True)
df_sorted['Date'].unique()

<DatetimeArray>
['2015-01-04 00:00:00', '2015-01-11 00:00:00', '2015-01-18 00:00:00',
 '2015-01-25 00:00:00', '2015-02-01 00:00:00', '2015-02-08 00:00:00',
 '2015-02-15 00:00:00', '2015-02-22 00:00:00', '2015-03-01 00:00:00',
 '2015-03-08 00:00:00',
 ...
 '2018-01-21 00:00:00', '2018-01-28 00:00:00', '2018-02-04 00:00:00',
 '2018-02-11 00:00:00', '2018-02-18 00:00:00', '2018-02-25 00:00:00',
 '2018-03-04 00:00:00', '2018-03-11 00:00:00', '2018-03-18 00:00:00',
 '2018-03-25 00:00:00']
Length: 169, dtype: datetime64[ns]

In [15]:
#We count to see the equally spaced and non-equally spaced, assuming the increment is 7 days (week)
from datetime import datetime
count_equal = 0
count_nonequal = 0
start_date = datetime(2015, 1, 4)
for index, row in df.iterrows():
    date_value = row['Date']
    date = pd.to_datetime(date_value)
    delta = date - start_date
    start_date = date
    
    # Check if the date is a multiple of seven
    if delta.days == 0:
        print("Same")
    elif delta.days % 7 == 0 :
        count_equal+=1
    else: 
        count_nonequal+=1

In [16]:
print(count_equal)
print(count_nonequal)

18249
0


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    


There is very clearly overlapping regions. Notice, Total US which may overlap with all the data from states and cities across the US. Similarly, California can overlap with SanFrancisco, SanDiego, Sacramento, etc. Thus, the regions are not all distinct. 

In [17]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023s/blob/main/lectures/19_time-series.ipynb), with some improvements.

In [18]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [19]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [20]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [21]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

### 1.4 `AveragePrice` baseline 

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    

Results are below.

In [22]:
from sklearn.metrics import r2_score
train_r2 = None
y_pred = df_train['AveragePrice']
y_actual = df_train['AveragePriceNextWeek']
train_r2 = r2_score(y_actual, y_pred)
train_r2

0.8285800937261841

In [23]:
test_r2 = None
y_pred_test = df_test['AveragePrice']
y_actual_test = df_test['AveragePriceNextWeek']
test_r2 = r2_score(y_actual_test, y_pred_test)
test_r2

0.7631780188583048

In [24]:
grader.check("q1.4")

q1.4 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.3
    


We've experimented with a few approachs below:

- Ridge score dropping Date. This produced the test_score 0.8 

- Ridge encoding Date as a number (days since). This produced a test_score of 0.75, so we did worse than simply dropping Date. 

- Ridge encoding in the data Months. This also produced a a test_score of 0.8. 

So far it seems the best score we get is 0.8. 

In [25]:
#Preprocessing: 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15441 entries, 0 to 18222
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  15441 non-null  datetime64[ns]
 1   AveragePrice          15441 non-null  float64       
 2   Total Volume          15441 non-null  float64       
 3   4046                  15441 non-null  float64       
 4   4225                  15441 non-null  float64       
 5   4770                  15441 non-null  float64       
 6   Total Bags            15441 non-null  float64       
 7   Small Bags            15441 non-null  float64       
 8   Large Bags            15441 non-null  float64       
 9   XLarge Bags           15441 non-null  float64       
 10  type                  15441 non-null  object        
 11  year                  15441 non-null  int64         
 12  region                15441 non-null  object        
 13  AveragePriceNextWeek 

In [26]:
df_train.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99


In [27]:
numeric_features = [
    "AveragePrice",
    "Total Volume", 
    "4046", 
    "4225", 
    "4770",
    "Total Bags", 
    "Small Bags", 
    "Large Bags", 
    "XLarge Bags"
]
categorical_features = [
    "type",
    "region", 
    "year" #Notice we let year be a categorical feature for now
]
drop_features = ["Date"] #We will first start by dropping Date
target = ["AveragePriceNextWeek"]

In [28]:
#Code taken from Lecture 19: 
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
def preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features,
    target
):

    all_features = set(numeric_features + categorical_features + drop_features + target)
    if set(train_df.columns) != all_features:
        print("Missing columns", set(train_df.columns) - all_features)
        print("Extra columns", all_features - set(train_df.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
         StandardScaler()
    )
    categorical_transformer = make_pipeline(
        OneHotEncoder(handle_unknown="ignore", sparse=False)
    )

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(train_df)
    ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(
        preprocessor.transform(train_df), index=train_df.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(test_df), index=test_df.index, columns=new_columns
    )

    y_train = train_df["AveragePriceNextWeek"]
    y_test = test_df["AveragePriceNextWeek"]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [29]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features,
    categorical_features,
    drop_features, target
)

C:\Users\sebas\miniconda3\envs\cpsc330\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
#Let us try the score and coefficents with Ridge (taken from lecture 19)
def score_ridge_print(preprocessor, train_df, y_train, test_df, y_test, X_train_enc):
    ridge_pipe = make_pipeline(preprocessor, Ridge())
    ridge_pipe.fit(train_df, y_train)
    print("Train score: {:.2f}".format(ridge_pipe.score(train_df, y_train)))
    print("Test score: {:.2f}".format(ridge_pipe.score(test_df, y_test)))

In [31]:
score_ridge_print(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

C:\Users\sebas\miniconda3\envs\cpsc330\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Train score: 0.85
Test score: 0.80


In [32]:
import warnings

warnings.filterwarnings('ignore')

In [33]:
#Let us cross validate on ridge with each location
ridge_pipe = make_pipeline(preprocessor, Ridge())
def cross_val_score_loc(train_df_loc):
    # print(train_df_loc['Location'].unique())
    return cross_val_score(
        ridge_pipe, train_df_loc, train_df_loc["AveragePriceNextWeek"], cv=TimeSeriesSplit()).mean()

location_results = df_train.groupby("region").apply(cross_val_score_loc)

In [34]:
location_results

region
Albany                 -6.358308
Atlanta                -3.732346
BaltimoreWashington    -0.052735
Boise                  -0.006967
Boston                 -3.294525
BuffaloRochester       -4.094153
California              0.554478
Charlotte              -0.814850
Chicago                 0.281595
CincinnatiDayton        0.319135
Columbus                0.175752
DallasFtWorth           0.328910
Denver                  0.068855
Detroit                -0.952585
GrandRapids             0.139200
GreatLakes              0.060944
HarrisburgScranton      0.070255
HartfordSpringfield    -1.146132
Houston                 0.689285
Indianapolis            0.260629
Jacksonville           -0.320973
LasVegas               -1.691338
LosAngeles              0.500639
Louisville              0.223565
MiamiFtLauderdale     -26.756942
Midsouth                0.066331
Nashville               0.106607
NewOrleansMobile       -1.968950
NewYork                -0.470601
Northeast               0.287141
Nor

### Encoding date/time as feature(s)
- Let us encode the date as a feature, not a dropped feature

In [35]:
#ENCODING DATE AS A NUMBER
first_day = df_train["Date"].min()
df_train = df_train.assign(
    Days_since=df_train["Date"].apply(lambda x: (x - first_day).days)
)
df_test = df_test.assign(
    Days_since=df_test["Date"].apply(lambda x: (x - first_day).days)
)

In [36]:
df_train.sort_values(by="Date").head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,Days_since
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,0
13858,2015-01-04,0.94,461607.33,244152.26,165299.33,15302.75,36852.99,30884.29,5595.00,373.7,conventional,2015,SanDiego,0.82,0
1352,2015-01-04,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015,Boston,1.10,0
13689,2015-01-04,1.33,9213.49,3727.52,4327.52,0.00,1158.45,1158.45,0.00,0.0,organic,2015,Sacramento,1.27,0
13520,2015-01-04,1.05,430138.88,110693.69,270107.61,9737.50,39600.08,39600.08,0.00,0.0,conventional,2015,Sacramento,1.09,0


In [37]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features + ["Days_since"],
    categorical_features,
    drop_features,
    target
)

In [38]:
score_ridge_print(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.85
Test score: 0.75


In [39]:
#ONE HOT ENCODING OF MONTH
df_train = df_train.assign(
    Month=df_train["Date"].apply(lambda x: x.month_name())
)  # x.month_name() to get the actual string
df_test = df_test.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))

In [40]:
# To ensure correct month ordering
months_cat = pd.CategoricalDtype(ordered=True,
    categories=['January', 'February', 'March', 'April', 'May', 'June',
                'July', 'August', 'September', 'October', 'November', 'December'])

df_train["Month"] = df_train["Month"].astype(months_cat)
df_test["Month"] = df_test["Month"].astype(months_cat)

In [41]:
df_train = df_train.drop(columns=["Days_since"])
df_test = df_test.drop(columns=["Days_since"])

In [42]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train, df_test, 
    numeric_features, 
    categorical_features + ["Month"], 
    drop_features,
    target
)

In [43]:
score_ridge_print(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.85
Test score: 0.80


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

### 2.1 Time series


1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    


1. An example of data that would have unequally spaced time points is any data that records the time at which an event occured, that is not in regular intervals. Some example like this could be recording climate events. 

2. The lagged version would struggle with unequally spaced time points. This is because to have some sort of temporal effect on that data (to predict the next in succession) we would prefer data that would be in regular time intervals. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis


1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2


1. Because in simply "binarizing" the problem we loose much of the information that produces different cases. We are really interested in the time until a customer has churned or not churned, not simply whether they have or have not.

2. We don't have enough information to answer that. It may be the case that the survival probability is quite low and thus customer A may leave the service first. Alternatively, the survival probability may depreciate with time, and so customer B may leave the service first. 

3. Given a Kaplan-Meier curve a horizontal curve represents a longer period at which the customer stays with the service. That is, with more flat periods we can interpret that the as being a better curve than one that is more vertical in steps. 

<!-- END QUESTION -->

<br><br>

![](img/eva-well-done.png)